# Alert Production Data Quality Report for {{params.instrument}} on {{ params.date }}

In [ ]:
# Parameters.  Set defaults here.

date = "2024-09-05"
instrument = "LATISS"

In [ ]:
day_obs = date.replace("-", "")  # day_obs is a string in the EFD
day_obs

In [ ]:
match instrument:
    case "LATISS":
        sal_index = 2
        n_detector = 1
    case "LSSTComCamSim":
        sal_index = 3
        n_detector = 9
    case _:
        logger.error(f"Unknown instrument {instrument}")

In [ ]:
# https://rtn-045.lsst.io/#colorblind-friendly-plots
plot_filter_colors = {
    "u": "#56b4e9",
    "g": "#008060",
    "r": "#ff4000",
    "i": "#850000",
    "z": "#6600cc",
    "y": "#000000",
}

1.3.14 Level 1 Data Quality Report Definition

ID: DMS-REQ-0097 (Priority: 1a)

Specification: The DMS shall produce a Level 1 Data Quality Report that contains indicators
of data quality that result from running the DMS pipelines, including at least: Photometric
zero point vs. time for each utilized filter; Sky brightness vs. time for each utilized filter; seeing
vs. time for each utilized filter; PSF parameters vs. time for each utilized filter; detection
efficiency for point sources vs. mag for each utilized filter.

Discussion: The seeing report is intended as a broad-brush measure of image quality. The
PSF parameters provide more detail, as they include asymmetries and field location
dependence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst_efd_client import EfdClient

As of 9 Oct 24, `lsst.prompt` is only available in usdfdev while we await authentication. 

In [ ]:
client = EfdClient("usdfdev_efd", db_name="lsst.prompt")

In [ ]:
# waiting on input in #sasquatch_support
# client = EfdClient("usdf_efd", db_name="lsst.prompt")

In [ ]:
await client.get_topics()

In [ ]:
await client.get_fields("lsst.prompt.prod.initialPviSummaryMetrics")

In [ ]:
# Testing to find good dates
# query = f"""SELECT * FROM "lsst.prompt.prod.initialPviSummaryMetrics" WHERE "day_obs" = '{day_obs}' AND "instrument" = '{instrument}' """
# print(query)
# initialPviSummaryMetrics = await client.influx_client.query(query)
# initialPviSummaryMetrics

In [ ]:
query = f"""SELECT * FROM "lsst.prompt.prod.initialPviSummaryMetrics" WHERE "instrument" = '{instrument}' """
print(query)
initialPviSummaryMetrics = await client.influx_client.query(query)
initialPviSummaryMetrics

In [ ]:
initialPviSummaryMetrics.columns

In [ ]:
def plot_metric_vs_time_by_filter(df, column):
    plt.figure()
    for filt in plot_filter_colors.keys():
        wf = df["band"] == filt

        if np.sum(wf):
            plt.plot(
                df.loc[wf].index,
                df.loc[wf, column],
                ".",
                color=plot_filter_colors[filt],
                label=filt,
            )

    plt.legend()
    plt.xlabel("Time")
    plt.ylabel(column)

In [ ]:
cols_to_plot = [
    "zeroPoint",
    "skyBg",
    "skyNoise",
    "psfSigma",
    "psfIxx",
    "psfIxy",
    "psfIyy",
    "psfArea",
]

In [ ]:
for col in cols_to_plot:
    plot_metric_vs_time_by_filter(initialPviSummaryMetrics, col)

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "zeroPoint")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "skyBg")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "skyNoise")

In [ ]:
plot_metric_vs_time_by_filter(initialPviSummaryMetrics, "psfSigma")

In [ ]:
await client.get_fields("lsst.prompt.numDiaSourcesAll")

https://docs.influxdata.com/influxdb/v1/query_language/explore-data/#the-where-clause

> Single quote string field values in the WHERE clause. Queries with unquoted string field values or double quoted string field values will not return any data and, in most cases, will not return an error.

In [ ]:
query = f"""SELECT * FROM "lsst.prompt.numDiaSourcesAll" WHERE "day_obs" = '{day_obs}' AND "instrument" = '{instrument}' """
print(query)
numDiaSourcesAll = await client.influx_client.query(query)
numDiaSourcesAll

In [ ]:
numDiaSourcesAll.columns

In [ ]:
numDiaSourcesAll.iloc[0]

In [ ]:
# help(client.select_time_series)

In [ ]:
for filt in plot_filter_colors.keys():
    wf = numDiaSourcesAll["band"] == filt

    if np.sum(wf):
        plt.plot(
            numDiaSourcesAll.loc[wf].index,
            numDiaSourcesAll.loc[wf, "numDiaSourcesAll"],
            ".",
            color=plot_filter_colors[filt],
            label=filt,
        )

plt.legend()
plt.xlabel("Time")
plt.ylabel("Number of DiaSources per detector")